<a href="https://colab.research.google.com/github/DemoySegment/dl-miniproject-resnet/blob/main/dl_miniproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchsummary
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as functional
import torchvision.models as models


In [2]:
# output_size = (input_size + 2*padding - kernel)/stride + 1 
class BuildingBlock(nn.Module):
   
    def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1, expansion=1):
      """
      This class is for building a resnet block. In each block various 
      convolution layers will be connected to each other with a batctnorm layer and a relu activation between.
      Skip connection will be built between the input of the first layer and the input of the last layer, 
      that is to add the input of the block to the output of the last batctnorm layer.
      Size of the two inputs of skip connection should be pay attention to.

      :param in_channels: the number of input channels of the whole block. Since block will repeat several times, 
      lets say a block with with a input channels of 64 and output channels of 128, the next time going
      through the block need a input channels of 128.

      :param intermediate_channels: the number of output channels of conv layers in the block. 
      Since channels always expand, the output channels of the block will be the expansion * intermediate_channels.

      :param identity_downsample: a model to deal with skip connection problem. this model should have a conv layer and
      a batchnorm layer. In the next iteration of same block, the input channels may not be consist with the output of the 
      last batchnorm output, therefore we need the parameter to help change x's channels.
      :type identity_downsample: nn.Module

      :paran stride: if stride>1 for one conv layer in each same block in iteration, then the size of the images will be 
      decreased for block_num times, which is not what we want. Therefore, for iterations of the the same block, only one layer
      in one of the block will have a stride that reduce the size of the image.
      """

      super().__init__()
      
      #expansion rate, the output channels of the block will be the expansion * intermediate_channels
      self.expansion = expansion
      self.conv1 = nn.Conv2d(
          in_channels,
          intermediate_channels,
          kernel_size=3,
          stride=1,
          padding=1,
          bias=False,
      )
      self.bn1 = nn.BatchNorm2d(intermediate_channels)
      self.conv2 = nn.Conv2d(
          intermediate_channels,
          intermediate_channels * self.expansion,
          kernel_size=3,
          stride=stride,
          padding=1,
          bias=False,
      )
      self.bn2 = nn.BatchNorm2d(intermediate_channels * self.expansion)
      self.relu = nn.ReLU()
      self.identity_downsample = identity_downsample
      self.stride = stride
      self.in_channels = in_channels
      self.intermediate_channels = intermediate_channels

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layerNums, image_channels, start_channels, num_classes):
        super(ResNet, self).__init__()
        # head layers
        self.in_channels = start_channels
        self.conv1 = nn.Conv2d(
            image_channels, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # recursion block layers
        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1, self.in_channels = self._make_block(
            BuildingBlock, layerNums[0], intermediate_channels=64, in_channels=self.in_channels, stride=1
        )
        self.layer2, self.in_channels = self._make_block(
            BuildingBlock, layerNums[1], intermediate_channels=128, in_channels=self.in_channels, stride=2
        )
        self.layer3, self.in_channels = self._make_block(
            BuildingBlock, layerNums[2], intermediate_channels=256, in_channels=self.in_channels, stride=2
        )
        self.layer4, self.in_channels = self._make_block(
            BuildingBlock, layerNums[3], intermediate_channels=512, in_channels=self.in_channels, stride=2
        )


        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

        self.layerNums = layerNums

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        #x = functional.softmax(x, dim=0)
        return x

    # for resnet18, expansion === 1
    def _make_block(self, block, num_layers, intermediate_channels, in_channels, stride, expansion=1):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        # it is used at the end of first iteration of each block
        if stride != 1 or in_channels != intermediate_channels*expansion:
          identity_downsample = nn.Sequential(
                  nn.Conv2d(
                      in_channels,
                      intermediate_channels*expansion,
                      kernel_size=1,
                      stride=stride,
                      bias=False,
                  ),
                  nn.BatchNorm2d(intermediate_channels*expansion),
              )

        layers.append(
            block(in_channels, intermediate_channels, stride=stride, identity_downsample=identity_downsample, expansion=expansion)
        )

       
        in_channels = intermediate_channels*expansion

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_layers - 1):
            layers.append(block(in_channels, intermediate_channels, expansion=expansion))
        
        return nn.Sequential(*layers), in_channels
    
    def to_string(self):
      print('current model status:')
      print('parameter numbers: {}'.format(sum(p.numel() for p in self.parameters() if p.requires_grad)))
      print('block numbers: {}'.format(self.layerNums))

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet(BuildingBlock, [2,1,1,1], 3, 32, 10)
model = model.to(device)
torchsummary.summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]           4,704
       BatchNorm2d-2           [-1, 32, 16, 16]              64
              ReLU-3           [-1, 32, 16, 16]               0
         MaxPool2d-4             [-1, 32, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          18,432
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
           Conv2d-10             [-1, 64, 8, 8]           2,048
      BatchNorm2d-11             [-1, 64, 8, 8]             128
             ReLU-12             [-1, 64, 8, 8]               0
    BuildingBlock-13             [-1, 64, 8, 8]               0
           Conv2d-14             [-1, 6

In [4]:
def accuracy(y_pred, y):
  predict = torch.argmax(y_pred, dim=1)
  acc = torch.sum(predict == y) / y.shape[0]
  return acc

In [5]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0

    #set the model in training mode
    model.train()

    for(x, y) in iterator:
      x = x.to(device)
      y = y.to(device)

      y_pred = model(x)
      
      loss = criterion(y_pred, y)
      acc = accuracy(y_pred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      epoch_loss += loss
      epoch_acc += acc
      

        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [6]:
def evaluate(model, iterator, criterion):
    

    epoch_loss = 0
    epoch_acc = 0

    #set the model in evaluation mode
    model.eval()
    with torch.no_grad():
      for(x, y) in iterator:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = criterion(y_pred, y)
        acc = accuracy(y_pred, y)
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [7]:
def run_epoches(epoch_num, model, optimizer, criterion, trainloader, testloader):
  best_valid_acc = float(0)
  print("start running")
  for epoch in range(N_EPOCHS):
      print(' --epoch {}'.format(epoch))
      print(" --start training--")
      train_loss, train_acc = train(model, trainloader, optimizer, criterion)
      print(" --start validing--")
      valid_loss, valid_acc = evaluate(model, testloader, criterion)
      if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc

      
      print(f'  \tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'  \t Val. Loss: {valid_loss:.3f} |  Val Acc: {valid_acc*100:.2f}%')
      print(f'  Current best Val Acc: {best_valid_acc}')
      torch.cuda.empty_cache()
  print("--end running")
  return best_valid_acc

In [ ]:
N_EPOCHS = 10
criterion = nn.CrossEntropyLoss()

criterion = criterion.to(device)

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

lr_candidates = torch.linspace(0.001, 0.1, 30)

batch_size_candidates = [64, 128, 256]

best_result = (0,0,0)
for lr in lr_candidates:
  for batch_size in batch_size_candidates:
    model = ResNet(BuildingBlock, [2,1,1,1], 3, 32, 10)
    model = model.to(device)
    print("--------------lr={}, batch_size={}, start-------------".format(lr, batch_size))
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr.item())

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)


    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    result = run_epoches(N_EPOCHS, model, optimizer, criterion, trainloader, testloader)
    print("--------------lr={}, batch_size={}, result={}-------------".format(lr, batch_size, result))
    if result > best_result[2]:
      best_result = (lr, batch_size, result)
    
print(best_result)

--------------lr=0.0010000000474974513, batch_size=64, start-------------
Files already downloaded and verified
Files already downloaded and verified
start running
 --epoch 0
 --start training--
 --start validing--
  	Train Loss: 1.326 | Train Acc: 52.22%
  	 Val. Loss: 1.081 |  Val Acc: 61.70%
  Current best Val Acc: 0.6170382499694824
 --epoch 1
 --start training--
 --start validing--
  	Train Loss: 0.965 | Train Acc: 65.90%
  	 Val. Loss: 0.896 |  Val Acc: 68.47%
  Current best Val Acc: 0.6847133636474609
 --epoch 2
 --start training--
 --start validing--
  	Train Loss: 0.796 | Train Acc: 72.00%
  	 Val. Loss: 0.808 |  Val Acc: 71.79%
  Current best Val Acc: 0.7178543210029602
 --epoch 3
 --start training--
 --start validing--
  	Train Loss: 0.679 | Train Acc: 76.32%
  	 Val. Loss: 0.753 |  Val Acc: 73.96%
  Current best Val Acc: 0.7396497130393982
 --epoch 4
 --start training--
 --start validing--
  	Train Loss: 0.580 | Train Acc: 79.69%
  	 Val. Loss: 0.720 |  Val Acc: 75.79%
  Cu

In [ ]:
BATCH_SIZE = 128
